<a href="https://colab.research.google.com/github/joshuaalpuerto/ML-guide/blob/main/RAG_hybrid_Mistral_7B_Instruct.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Loading a GPTQ quantized model requires optimum (`pip install optimum`) and auto-gptq library (`pip install auto-gptq`)
!pip install -q optimum --progress-bar off
!pip install -q auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/  --progress-bar off # Use cu117 if on CUDA 11.7
# We need specific transformer to make mistral work
!pip install -q git+https://github.com/huggingface/transformers.git@72958fcd3c98a7afdc61f953aa58c544ebda2f79 --progress-bar off

!pip install -qU langchain Faiss-gpu sentence-transformers
# !pip install -qU trl Py7zr
!pip install -q rank_bm25
!pip install -q PyPdf

In [ ]:
import langchain
from langchain.embeddings import CacheBackedEmbeddings,HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.retrievers import BM25Retriever,EnsembleRetriever
from langchain.document_loaders import PyPDFLoader,DirectoryLoader, JSONLoader
from langchain.llms import CTransformers
from langchain.cache import InMemoryCache
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.schema import prompt
from langchain.chains import RetrievalQA
from langchain.callbacks import StdOutCallbackHandler
from langchain import PromptTemplate
#
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline

In [ ]:
# changes

In [ ]:
#connect to google drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Define the metadata extraction function.
def metadata_func(record: dict, metadata: dict) -> dict:

    metadata["country"] = record.get("country")
    metadata["answer"] = record.get("answer")

    return metadata


loader = JSONLoader(
    file_path='/content/drive/MyDrive/datasets/qna-clean.json',
    jq_schema='.',
    content_key="question",
    metadata_func=metadata_func
)

faq_docs = loader.load()

In [ ]:
# In our implementation we have used uses the local file system for storing embeddings and FAISS vector store for retrieval.
# store = LocalFileStore("./cache/")

# We can also set up inmemory cache
# NOTE: we used this as we are more familiar with it
store = InMemoryStore()

In [ ]:
from langchain.llms import CTransformers
config = {'max_new_tokens': 1024, 'temperature': 0}
llm = CTransformers(model='TheBloke/Mistral-7B-Instruct-v0.1-GGUF',
                    model_file="mistral-7b-instruct-v0.1.Q4_K_M.gguf",
                    config=config)

In [ ]:
embed_model_id="thenlper/gte-large"

# Under the hood HuggingFaceEmbeddings is using sentence-transformer
core_embeddings_model = HuggingFaceEmbeddings(model_name=embed_model_id)

# Here we will leverage a CacheBackedEmbeddings to prevent us from re-embedding similar queries over and over again.
embedder = CacheBackedEmbeddings.from_bytes_store(core_embeddings_model,
                                                  store,
                                                  namespace=embed_model_id)

In [ ]:
# Create VectorStore
vectorstore = FAISS.from_documents(faq_docs, embedder)

In [ ]:
#Step 05: Split the Extracted Data into Text Chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=20)

text_chunks = text_splitter.split_documents(data)


In [ ]:
len(text_chunks)

23

In [ ]:
#get the third chunk
text_chunks[2]

Document(page_content='Econometric Computing with HC and HAC Covariance\nMatrix Estimators\nAchim Zeileis\nUniversit ¨at Innsbruck\nAbstract\nThis introduction to the Rpackagesandwich is a (slightly) modiﬁed version of Zeileis\n(2004), published in the Journal of Statistical Software . A follow-up paper on object\nobject-oriented computation of sandwich estimators is available in ( Zeileis 2006b ).\nData described by econometric models typically contains autocorrel ation and/or het-\neroskedasticity of unknown form and for inference in such models it is essential to use\ncovariance matrix estimators that can consistently estimate the covar iance of the model\nparameters. Hence, suitable heteroskedasticity-consistent (HC) an d heteroskedasticity\nand autocorrelation consistent (HAC) estimators have been receiving attention in the\neconometric literature over the last 20 years. To apply these estimat ors in practice,\nan implementation is needed that preferably translates the conceptu al

In [ ]:
#Step 06:Downlaod the Embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [ ]:
#Step 08: Create Embeddings for each of the Text Chunk
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)

In [ ]:
#connect to google drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#Import Model
llm = LlamaCpp(
    streaming = True,
    model_path="/content/drive/MyDrive/Model/mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    temperature=0.75,
    top_p=1,
    verbose=True,
    n_ctx=4096
)

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 0 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 


In [ ]:
qa = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vector_store.as_retriever(search_kwargs={"k": 2}))

In [ ]:
query = "What is linear regression model"

In [ ]:
qa.run(query)

' The linear regression model is a statistical method for modeling the relationship between a dependent variable and one or more independent variables, commonly referred to as predictors or explanatory variables. In this paper, the linear regression model is used as a starting point to derive estimators for the variance-covariance matrix of the errors in certain structural change models.'

In [ ]:
import sys

while True:
  user_input = input(f"Input Prompt: ")
  if user_input == 'exit':
    print('Exiting')
    sys.exit()
  if user_input == '':
    continue
  result = qa({'query': user_input})
  print(f"Answer: {result['result']}")

Llama.generate: prefix-match hit


Answer:  It is a statistical method used to analyze the linear relationship between two or more variables. In this case, it is used to estimate the coefficients of a linear regression model.


Llama.generate: prefix-match hit


Answer:  sandwich provides various functions for estimating the covariance matrix. The most commonly used ones are vcovHAC and vcovHC. These functions can take different weighting schemes, including kernel-based HAC estimation with automatic bandwidth selection based on weightsAndrews andbwAndrews. Other functions like weightsLumley provide different weighting schemes, such as truncated and smoothed weights, which are useful in certain applications. In econometric analyses, these functions are used to compute partial t-tests for assessing the significance of a parameter. The choice of estimator depends on the presence of heteroscedasticity and autocorrelation in the errors.
